In [22]:
import pandas as pd

# Replace 'your_file.json' with the path to your JSON file
df = pd.read_csv(r"C:\Users\ASUS\Downloads\SC4001 Neural Network project\IMDB Dataset.csv")
print(len(df))
print(df.head())
df["sentiment"] = df["sentiment"].apply(lambda x: 1 if x == "positive" else 0)
print(df.head())
df = df.reset_index(drop=True)

50000
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                              review  sentiment
0  One of the other reviewers has mentioned that ...          1
1  A wonderful little production. <br /><br />The...          1
2  I thought this was a wonderful way to spend ti...          1
3  Basically there's a family where a little boy ...          0
4  Petter Mattei's "Love in the Time of Money" is...          1


In [23]:
from sklearn.model_selection import train_test_split

# Train+val / test split
train_val_df, test_df = train_test_split(df, test_size=0.98, random_state=42, stratify=df["sentiment"])

# train / val split
train_df, val_df = train_test_split(train_val_df, test_size=0.5, random_state=42, stratify=train_val_df["sentiment"])

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Train size: 500
Validation size: 500
Test size: 49000


In [17]:
from itertools import islice

input_path = r"C:\Users\ASUS\Downloads\SC4001 Neural Network project\yelp_academic_dataset_review.json"
output_path = r"C:\Users\ASUS\Downloads\SC4001 Neural Network project\yelp_sample.json"

with open(input_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', encoding='utf-8') as outfile:
    for line in islice(infile, 50000):
        outfile.write(line)

In [ ]:
import pandas as pd

# Replace 'your_file.json' with the path to your JSON file
yelp_df = pd.read_json(r"C:\Users\ASUS\Downloads\SC4001 Neural Network project\yelp_sample.json",lines=True)
print(len(yelp_df))
# print(df.head())
# print(yelp_df.columns)
yelp_df = yelp_df[yelp_df["stars"]!=3]
yelp_df['sentiment'] = yelp_df['stars'].apply(lambda x: 1 if x >= 4 else 0)

print(yelp_df["stars"].value_counts())
print(yelp_df["sentiment"].value_counts())

yelp_df = yelp_df[["text","sentiment"]]

50000
stars
5    22220
4    12721
1     5379
2     4003
Name: count, dtype: int64
sentiment
1    34941
0     9382
Name: count, dtype: int64
Index(['text', 'sentiment'], dtype='object')


In [30]:
yelp_test_texts = yelp_df["text"]
yelp_test_labels = yelp_df["sentiment"]
yelp_data = IMDBDataset(yelp_test_texts, yelp_test_labels)
yelp_loader = DataLoader(yelp_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
print(evaluate(model,yelp_loader))

(0.6993885792929179, 0.8019972656482197)


In [24]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
# from datasets import load_dataset
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
import re
import copy

train_texts = train_df["review"]
train_labels = train_df["sentiment"]
val_texts = val_df["review"]
val_labels = val_df["sentiment"]
test_texts = test_df["review"]
test_labels = test_df["sentiment"]

def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.split()

# def tokenize(text, max_len=300):
#     text = text.lower()
#     text = re.sub(r"[^a-z0-9\s]", "", text)
#     return text.split()[:max_len]

counter = Counter()
for text in train_texts:
    counter.update(tokenize(text))
print(f"counterlen: {len(counter)}")
vocab_size = 20000
vocab = {word: idx + 2 for idx, (word, _) in enumerate(counter.most_common(vocab_size))}
vocab["<pad>"] = 0
vocab["<unk>"] = 1

def numericalize(tokens):
    return [vocab.get(token, vocab["<unk>"]) for token in tokens]

class IMDBDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = [torch.tensor(numericalize(tokenize(t))) for t in texts]
        self.labels = torch.tensor(labels.tolist())
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

def collate_batch(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=vocab["<pad>"])
    return texts_padded, torch.tensor(labels)

train_data = IMDBDataset(train_texts, train_labels)
val_data = IMDBDataset(val_texts, val_labels)
test_data = IMDBDataset(test_texts, test_labels)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False, collate_fn=collate_batch)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_batch)

counterlen: 13861


In [26]:
from sklearn.metrics import accuracy_score, f1_score

def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for texts, labels in dataloader:
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader):
    model.eval()
    preds, labels_all = [], []
    with torch.no_grad():
        for texts, labels in dataloader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)
            preds.extend(predictions.cpu().tolist())
            labels_all.extend(labels.cpu().tolist())
    acc = accuracy_score(labels_all, preds)
    f1 = f1_score(labels_all, preds)
    return acc, f1

# Simple RNN

In [27]:
# class SimpleRNN(nn.Module):
#     def __init__(self, vocab_size, embed_dim, hidden_size, output_size, num_layers=1):
#         super(SimpleRNN, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, embed_dim)
#         self.rnn = nn.RNN(embed_dim, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, output_size)
#         # self.sigmoid = nn.Sigmoid()


#     def forward(self, x):
#         x = self.embedding(x)
#         out, _ = self.rnn(x)
#         out = self.fc(out[:, -1, :]) # last hidden state
#         return out

class SimpleRNN(nn.Module):
    def __init__(self, vocab_size,embed_dim, hidden_size, output_size, method="last", num_layers=1, dropout=0):
        super(SimpleRNN, self).__init__()
        # self.embedding_transform = nn.Linear(embed_dim, embed_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_size, num_layers, batch_first=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.method = method

    def forward(self, x):
        # embeds = self.embedding_transform(x)
        embeds = self.embedding(x)
        out, _ = self.rnn(embeds)
        if self.method == "last":
            out = self.dropout(self.fc(out[:, -1, :]))
        elif self.method == "avg":
            out = self.dropout(self.fc(out.mean(dim=1)))
        elif self.method == "max":
            out, _ = out.max(dim=1)
            out = self.dropout(self.fc(out))
        return out

model = SimpleRNN(vocab_size=len(vocab), embed_dim=100, hidden_size=128, output_size=2, method="avg").to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=10e-3)

best_acc = 0.0
best_model_state = None

for epoch in range(50):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: loss={train_loss:.4f}, acc={acc:.4f}, f1={f1:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"New best accuracy: {best_acc:.4f} at epoch {epoch+1}, saving model.")

Epoch 1: loss=0.8398, acc=0.5080, f1=0.6694
New best accuracy: 0.5080 at epoch 1, saving model.
Epoch 2: loss=0.7135, acc=0.5120, f1=0.0469
New best accuracy: 0.5120 at epoch 2, saving model.
Epoch 3: loss=0.7262, acc=0.5040, f1=0.6612
Epoch 4: loss=0.7027, acc=0.5060, f1=0.6676
Epoch 5: loss=0.7369, acc=0.5000, f1=0.6667
Epoch 6: loss=0.7000, acc=0.5000, f1=0.6667
Epoch 7: loss=0.6828, acc=0.4780, f1=0.3040
Epoch 8: loss=0.7386, acc=0.5000, f1=0.0000
Epoch 9: loss=0.6989, acc=0.5020, f1=0.0812
Epoch 10: loss=0.6836, acc=0.5020, f1=0.1942
Epoch 11: loss=0.7216, acc=0.4880, f1=0.6246
Epoch 12: loss=0.6815, acc=0.4900, f1=0.6256
Epoch 13: loss=0.6719, acc=0.4880, f1=0.6086
Epoch 14: loss=0.6474, acc=0.5440, f1=0.2500
New best accuracy: 0.5440 at epoch 14, saving model.
Epoch 15: loss=0.6132, acc=0.4840, f1=0.5994
Epoch 16: loss=0.4870, acc=0.5840, f1=0.6498
New best accuracy: 0.5840 at epoch 16, saving model.
Epoch 17: loss=0.3792, acc=0.5840, f1=0.4439
Epoch 18: loss=0.3118, acc=0.6760,

In [25]:
model = SimpleRNN(vocab_size=len(vocab), embed_dim=100, hidden_size=128, output_size=2, method="avg").to(device)
# model.load_state_dict(torch.load(best_model_state))
model.load_state_dict(best_model_state)
model.eval()
print(evaluate(model, test_loader))

(0.7297959183673469, 0.7170456488288597)


# BiGRU RNN

In [34]:
class BiGRU(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, output_size, method="last", num_layers=1, dropout=0):
        super(BiGRU, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # BiGRU - note bidirectional=True
        self.gru = nn.GRU(embed_dim, hidden_size, num_layers, 
                          batch_first=True, bidirectional=True)
        
        # Linear layer input size is 2*hidden_size due to bidirectional
        self.fc = nn.Linear(2 * hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.method = method
        
    def forward(self, x):
        embeds = self.embedding(x)
        out, hidden = self.gru(embeds)  # out: (batch, seq_len, 2*hidden_size)
        
        if self.method == "last":
            out = self.dropout(self.fc(out[:, -1, :]))
        elif self.method == "avg":
            out = self.dropout(self.fc(out.mean(dim=1)))
        elif self.method == "max":
            out, _ = out.max(dim=1)
            out = self.dropout(self.fc(out))
            
        return out
    
model = BiGRU(vocab_size=len(vocab), embed_dim=100, hidden_size=128, output_size=2, method="last").to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=10e-3)

best_acc = 0.0
best_model_state = None

for epoch in range(50):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: loss={train_loss:.4f}, acc={acc:.4f}, f1={f1:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"New best accuracy: {best_acc:.4f} at epoch {epoch+1}, saving model.")

Epoch 1: loss=0.9805, acc=0.5040, f1=0.6640
New best accuracy: 0.5040 at epoch 1, saving model.
Epoch 2: loss=0.7053, acc=0.5000, f1=0.0385
Epoch 3: loss=0.6741, acc=0.5080, f1=0.6621
New best accuracy: 0.5080 at epoch 3, saving model.
Epoch 4: loss=0.4112, acc=0.5360, f1=0.2795
New best accuracy: 0.5360 at epoch 4, saving model.
Epoch 5: loss=0.1729, acc=0.5720, f1=0.6137
New best accuracy: 0.5720 at epoch 5, saving model.
Epoch 6: loss=0.0436, acc=0.5900, f1=0.6293
New best accuracy: 0.5900 at epoch 6, saving model.
Epoch 7: loss=0.0268, acc=0.5660, f1=0.6328
Epoch 8: loss=0.0295, acc=0.5960, f1=0.6145
New best accuracy: 0.5960 at epoch 8, saving model.
Epoch 9: loss=0.0100, acc=0.5920, f1=0.5904
Epoch 10: loss=0.0035, acc=0.6020, f1=0.5914
New best accuracy: 0.6020 at epoch 10, saving model.
Epoch 11: loss=0.0023, acc=0.5920, f1=0.5660
Epoch 12: loss=0.0018, acc=0.5940, f1=0.5690
Epoch 13: loss=0.0015, acc=0.5960, f1=0.5684
Epoch 14: loss=0.0013, acc=0.5960, f1=0.5665
Epoch 15: loss

# Bi RNN model

In [32]:
class BiLSTM(nn.Module):
    def __init__(self, vocab_size,embed_dim, hidden_size, output_size, method="last", num_layers=1, dropout=0):
        super(BiLSTM, self).__init__()
        # self.embedding_transform = nn.Linear(embed_dim, embed_dim)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # BiLSTM - note bidirectional=True
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, 
                           batch_first=True, bidirectional=True)
        
        # Linear layer input size is 2*hidden_size due to bidirectional
        self.fc = nn.Linear(2 * hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.method = method
        
    def forward(self, x):
        embeds = self.embedding(x)
        out, (hidden, cell) = self.lstm(embeds) 
        
        if self.method == "last":
            out = self.dropout(self.fc(out[:, -1, :]))
        elif self.method == "avg":
            out = self.dropout(self.fc(out.mean(dim=1)))
        elif self.method == "max":
            out, _ = out.max(dim=1)
            out = self.dropout(self.fc(out))
            
        return out

model = BiLSTM(vocab_size=len(vocab), embed_dim=100, hidden_size=128, output_size=2, method="avg").to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=10e-3)

best_acc = 0.0
best_model_state = None

for epoch in range(50):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model, val_loader)
    print(f"Epoch {epoch+1}: loss={train_loss:.4f}, acc={acc:.4f}, f1={f1:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"New best accuracy: {best_acc:.4f} at epoch {epoch+1}, saving model.")

Epoch 1: loss=0.7495, acc=0.5240, f1=0.5897
New best accuracy: 0.5240 at epoch 1, saving model.
Epoch 2: loss=0.6860, acc=0.5260, f1=0.6758
New best accuracy: 0.5260 at epoch 2, saving model.
Epoch 3: loss=0.6247, acc=0.5520, f1=0.6845
New best accuracy: 0.5520 at epoch 3, saving model.
Epoch 4: loss=0.3927, acc=0.6160, f1=0.7064
New best accuracy: 0.6160 at epoch 4, saving model.
Epoch 5: loss=0.1896, acc=0.7360, f1=0.7179
New best accuracy: 0.7360 at epoch 5, saving model.
Epoch 6: loss=0.0556, acc=0.6760, f1=0.5549
Epoch 7: loss=0.0210, acc=0.6860, f1=0.5964
Epoch 8: loss=0.0109, acc=0.6980, f1=0.6308
Epoch 9: loss=0.0057, acc=0.6920, f1=0.6207
Epoch 10: loss=0.0066, acc=0.6800, f1=0.5897
Epoch 11: loss=0.0054, acc=0.6980, f1=0.6326
Epoch 12: loss=0.0036, acc=0.7060, f1=0.6458
Epoch 13: loss=0.0035, acc=0.7080, f1=0.6490
Epoch 14: loss=0.0024, acc=0.7100, f1=0.6506
Epoch 15: loss=0.0021, acc=0.7120, f1=0.6488
Epoch 16: loss=0.0008, acc=0.7080, f1=0.6404
Epoch 17: loss=0.0007, acc=0.

# Regular LSTM model

In [ ]:


class LSTMSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        return self.fc(hidden)


model = LSTMSentiment(vocab_size=len(vocab), embed_dim=100, hidden_dim=128, output_dim=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=10e-3)

best_acc = 0.0
best_model_state = None

train_losses = []
val_accuracies = []
val_f1s = []

for epoch in range(200):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model, test_loader)
    
    # 🔹 Save metrics for plotting
    train_losses.append(train_loss)
    val_accuracies.append(acc)
    val_f1s.append(f1)
    
    print(f"Epoch {epoch+1}: loss={train_loss:.4f}, acc={acc:.4f}, f1={f1:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"New best accuracy: {best_acc:.4f} at epoch {epoch+1}, saving model.")

Epoch 1: Loss=0.7001 | Acc=0.4980 | F1=0.6613
Epoch 2: Loss=0.6853 | Acc=0.5020 | F1=0.0460
Epoch 3: Loss=0.6864 | Acc=0.5000 | F1=0.6640
Epoch 4: Loss=0.6870 | Acc=0.5020 | F1=0.0311
Epoch 5: Loss=0.6847 | Acc=0.5020 | F1=0.6631
Epoch 6: Loss=0.6799 | Acc=0.4980 | F1=0.0079
Epoch 7: Loss=0.6748 | Acc=0.4960 | F1=0.6585
Epoch 8: Loss=0.6757 | Acc=0.5000 | F1=0.6622
Epoch 9: Loss=0.6889 | Acc=0.5020 | F1=0.0235
Epoch 10: Loss=0.6717 | Acc=0.5020 | F1=0.6640
Epoch 11: Loss=0.6782 | Acc=0.4960 | F1=0.0079
Epoch 12: Loss=0.6720 | Acc=0.4960 | F1=0.0156
Epoch 13: Loss=0.6728 | Acc=0.4960 | F1=0.6595
Epoch 14: Loss=0.6714 | Acc=0.4960 | F1=0.6595
Epoch 15: Loss=0.6765 | Acc=0.5000 | F1=0.0385
Epoch 16: Loss=0.6781 | Acc=0.5000 | F1=0.6622
Epoch 17: Loss=0.6745 | Acc=0.5020 | F1=0.0311
Epoch 18: Loss=0.6790 | Acc=0.5000 | F1=0.6631
Epoch 19: Loss=0.6749 | Acc=0.5000 | F1=0.0234
Epoch 20: Loss=0.6774 | Acc=0.5000 | F1=0.6622
Epoch 21: Loss=0.6770 | Acc=0.4980 | F1=0.6613
Epoch 22: Loss=0.6716 

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# 📉 Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, label='Train Loss', marker='o')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# 📈 Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, val_accuracies, label='Validation Accuracy', marker='o', color='orange')
plt.title('Validation Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [20]:
from torchtext.vocab import GloVe
glove = GloVe(name="6B", dim=100)
embedding_dim = glove.dim

# Create embedding matrix
embedding_matrix = torch.zeros(len(vocab), embedding_dim)
for word, idx in vocab.items():
    if word in glove.stoi:
        embedding_matrix[idx] = glove[word]
    else:
        embedding_matrix[idx] = torch.randn(embedding_dim) * 0.05  # random init for OOV words

c:\Users\ASUS\anaconda3\envs\SC4001\Lib\site-packages\torchtext\vocab.py:432: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.itos, self.stoi, self.vectors, self.dim = to

In [ ]:


class LSTM_GLOVE(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, embeddings, freeze_embed=True):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=vocab["<pad>"])
        self.embedding.weight.data.copy_(embeddings)
        self.embedding.weight.requires_grad = not freeze_embed  # freeze or fine-tune
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, _) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        return self.fc(hidden)


# def train_epoch(model, dataloader, optimizer, criterion):
#     model.train()
#     total_loss = 0
#     for texts, labels in dataloader:
#         texts, labels = texts.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(texts)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     return total_loss / len(dataloader)

# def evaluate(model, dataloader):
#     model.eval()
#     preds, labels_all = [], []
#     with torch.no_grad():
#         for texts, labels in dataloader:
#             texts, labels = texts.to(device), labels.to(device)
#             outputs = model(texts)
#             predictions = torch.argmax(outputs, dim=1)
#             preds.extend(predictions.cpu().tolist())
#             labels_all.extend(labels.cpu().tolist())
#     acc = accuracy_score(labels_all, preds)
#     f1 = f1_score(labels_all, preds)
#     return acc, f1


model_frozen = LSTMSentiment(
    vocab_size=len(vocab),
    embed_dim=embedding_dim,
    hidden_dim=128,
    output_dim=2,
    embeddings=embedding_matrix,
    freeze_embed=True
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_frozen.parameters(), lr=5e-3)

print("==== Training with Frozen GloVe embeddings ====")
for epoch in range(50):
    loss = train_epoch(model_frozen, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model_frozen, test_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f} | Acc={acc:.4f} | F1={f1:.4f}")

# =====================================================
# 8. Phase 2: Fine-tune embeddings
# =====================================================
print("\n==== Fine-tuning GloVe embeddings ====")
model_finetuned = LSTMSentiment(
    vocab_size=len(vocab),
    embed_dim=embedding_dim,
    hidden_dim=128,
    output_dim=2,
    embeddings=embedding_matrix,
    freeze_embed=False
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_finetuned.parameters(), lr=5e-3)

for epoch in range(75):
    loss = train_epoch(model_finetuned, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model_finetuned, test_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f} | Acc={acc:.4f} | F1={f1:.4f}")

==== Training with Frozen GloVe embeddings ====
Epoch 1: Loss=0.6946 | Acc=0.5004 | F1=0.6616
Epoch 2: Loss=0.6931 | Acc=0.5012 | F1=0.6636
Epoch 3: Loss=0.6948 | Acc=0.4988 | F1=0.6645
Epoch 4: Loss=0.6927 | Acc=0.5028 | F1=0.6663
Epoch 5: Loss=0.6902 | Acc=0.5016 | F1=0.6634
Epoch 6: Loss=0.6866 | Acc=0.5048 | F1=0.0492
Epoch 7: Loss=0.6879 | Acc=0.5064 | F1=0.0434
Epoch 8: Loss=0.6943 | Acc=0.5028 | F1=0.6641
Epoch 9: Loss=0.6903 | Acc=0.5056 | F1=0.0463
Epoch 10: Loss=0.6863 | Acc=0.5056 | F1=0.0507
Epoch 11: Loss=0.6893 | Acc=0.5032 | F1=0.0387
Epoch 12: Loss=0.6872 | Acc=0.5016 | F1=0.6640
Epoch 13: Loss=0.6842 | Acc=0.5032 | F1=0.0297
Epoch 14: Loss=0.6834 | Acc=0.5016 | F1=0.6640
Epoch 15: Loss=0.6798 | Acc=0.5020 | F1=0.0401
Epoch 16: Loss=0.6860 | Acc=0.5024 | F1=0.6647
Epoch 17: Loss=0.6825 | Acc=0.5040 | F1=0.6661
Epoch 18: Loss=0.6809 | Acc=0.5016 | F1=0.6643
Epoch 19: Loss=0.6789 | Acc=0.5028 | F1=0.0342
Epoch 20: Loss=0.6805 | Acc=0.5040 | F1=0.0417
Epoch 21: Loss=0.6787

In [9]:
print(train_labels.unique())
print(train_labels.value_counts())

[1 0]
sentiment
1    1250
0    1250
Name: count, dtype: int64


In [11]:
unk_count = sum(1 for w in vocab if w not in glove.stoi)
print(f"OOV rate: {unk_count/len(vocab):.2%}")

OOV rate: 12.67%


In [13]:
texts, labels = next(iter(train_loader))
outputs = model_finetuned(texts.to(device))
print(outputs[:5])

tensor([[-6.2704e-01,  4.8618e-01],
        [-6.9359e-03, -1.8749e-05],
        [-6.9359e-03, -1.8749e-05],
        [-6.9359e-03, -1.8749e-05],
        [-6.9359e-03, -1.8749e-05]], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
def test(model, dataloader):
    model.eval()
    preds, labels_all = [], []
    with torch.no_grad():
        for texts, labels in dataloader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)
            preds.extend(predictions.cpu().tolist())
            labels_all.extend(labels.cpu().tolist())
    acc = accuracy_score(labels_all, preds)
    f1 = f1_score(labels_all, preds)
    return acc, f1

test_texts_2 = test_df["review"]
test_labels_2 = test_df["sentiment"]
test_data_2 = IMDBDataset(test_texts_2, test_labels_2)
test_loader_2 = DataLoader(test_data_2, batch_size=32, shuffle=False, collate_fn=collate_batch)

acc, f1 = test(model_finetuned, test_loader_2)

print(f" Acc={acc:.4f} | F1={f1:.4f}")

 Acc=0.7882 | F1=0.7871


In [ ]:
def test(model, dataloader, texts_list=None, label_names=None):
    model.eval()
    preds, labels_all = [], []
    wrong_predictions = []  # store (text, true, pred)

    with torch.no_grad():
        for batch_idx, (texts, labels) in enumerate(dataloader):
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)

            preds.extend(predictions.cpu().tolist())
            labels_all.extend(labels.cpu().tolist())

            # if you passed the original text list, we can recover them
            if texts_list is not None:
                batch_start = batch_idx * dataloader.batch_size
                batch_end = batch_start + len(labels)
                for i in range(len(labels)):
                    if predictions[i] != labels[i]:
                        wrong_predictions.append({
                            "text": texts_list[batch_start + i],
                            "true_label": (label_names[labels[i].item()] if label_names else labels[i].item()),
                            "pred_label": (label_names[predictions[i].item()] if label_names else predictions[i].item())
                        })

    acc = accuracy_score(labels_all, preds)
    f1 = f1_score(labels_all, preds)

    print(f"Acc={acc:.4f} | F1={f1:.4f}")
    print(f"Total wrong predictions: {len(wrong_predictions)}")

    # Print examples
    for w in wrong_predictions[:5]:  # show first 5
        print("─" * 60)
        print(f"True: {w['true_label']} | Pred: {w['pred_label']}")
        print(f"Text: {w['text']}")  # limit text length for readability

    return acc, f1, wrong_predictions

test_texts_2 = test_df["review"]
test_labels_2 = test_df["sentiment"]
test_data_2 = IMDBDataset(test_texts_2, test_labels_2)
test_loader_2 = DataLoader(test_data_2, batch_size=32, shuffle=False, collate_fn=collate_batch)

# acc, f1 = test(model_finetuned, test_loader_2)
label_names = {0: "negative", 1: "positive"}

acc, f1, wrong_preds = test(
    model_finetuned,
    test_loader_2,
    texts_list=test_df["review"].tolist(),
    label_names=label_names
)
print(f" Acc={acc:.4f} | F1={f1:.4f}")

In [21]:
import torch
import torch.nn as nn

class SkipLSTM(nn.Module):
    """
    Simpler LSTM with dynamic skip connection on top of nn.LSTM output.
    Skips are computed at each timestep based on input + LSTM hidden output.
    """

    def __init__(self, input_size, hidden_size, num_layers=1, gate_type="scalar", batch_first=True):
        super().__init__()
        assert gate_type in ("scalar", "vector"), "gate_type must be 'scalar' or 'vector'"
        self.hidden_size = hidden_size
        self.gate_type = gate_type
        self.batch_first = batch_first

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=batch_first)

        # Gate takes concatenation of input and LSTM output at each step
        gate_out_dim = 1 if gate_type == "scalar" else hidden_size
        self.gate = nn.Linear(input_size + hidden_size, gate_out_dim)

        # Optional: initialize gate bias so model starts with more updates than skips
        nn.init.constant_(self.gate.bias, -2.0)

    def forward(self, x, hx=None):
        """
        Args:
            x: (batch, seq_len, input_size)
            hx: optional (h0, c0)
        Returns:
            outputs: (batch, seq_len, hidden_size)
            (h_n, c_n): final states
        """
        lstm_out, (h_n, c_n) = self.lstm(x, hx)  # (B, T, H)

        # Compute skip gate for each timestep
        # Concatenate input and LSTM output → (B, T, input+hidden)
        gate_in = torch.cat([x, lstm_out], dim=-1)
        g = torch.sigmoid(self.gate(gate_in))  # (B, T, 1 or H)

        if self.gate_type == "scalar":
            g = g.expand_as(lstm_out)

        # Apply dynamic skip connection:
        # Smoothly mix previous timestep output and current LSTM output
        # Shift lstm_out by one timestep for "previous" context
        prev_out = torch.zeros_like(lstm_out)
        prev_out[:, 1:, :] = lstm_out[:, :-1, :]

        outputs = g * prev_out + (1 - g) * lstm_out
        return outputs, (h_n, c_n)


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_classes, pad_idx, embeddings=None, freeze_embed=False):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        if embeddings is not None:
            self.embedding.weight.data.copy_(embeddings)
            self.embedding.weight.requires_grad = not freeze_embed  # freeze or fine-tune
        self.encoder = SkipLSTM(embed_dim, hidden_size, gate_type="scalar", batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (batch, seq_len)
        embedded = self.embedding(x)                # (B, T, E)
        outputs, _ = self.encoder(embedded)         # (B, T, H)
        last_hidden = outputs[:, -1, :]             # take last timestep
        logits = self.fc(last_hidden)               # (B, num_classes)
        return logits
    
# hidden_size = 64
# input_size= len(vocab)
# model = LSTMClassifier(input_size, hidden_size, 2).to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [29]:
import copy
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for texts, labels in dataloader:
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)


def evaluate(model, dataloader):
    model.eval()
    preds, labels_all = [], []

    with torch.no_grad():
        for texts, labels in dataloader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)
            preds.extend(predictions.cpu().tolist())
            labels_all.extend(labels.cpu().tolist())

    acc = accuracy_score(labels_all, preds)
    f1 = f1_score(labels_all, preds, average="macro")
    return acc, f1


# =====================================================
# 3. Training setup
# =====================================================
vocab_size = len(vocab)
embed_dim = 100
hidden_size = 192
num_classes = 2  # binary sentiment classification
pad_idx = vocab["<pad>"]

model = LSTMClassifier(vocab_size, embed_dim, hidden_size, num_classes, pad_idx,embeddings=embedding_matrix).to(device)
# model = LSTMClassifier(vocab_size, embed_dim, hidden_size, num_classes, pad_idx,embeddings=None).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# =====================================================
# 4. Training loop
# =====================================================
best_acc = 0.0
best_model_state = None
for epoch in range(200):
    train_loss = train_epoch(model, train_loader, optimizer, criterion)
    acc, f1 = evaluate(model, test_loader)
    print(f"Epoch {epoch+1}: loss={train_loss:.4f}, acc={acc:.4f}, f1={f1:.4f}")
    if acc > best_acc:
        best_acc = acc
        best_model_state = copy.deepcopy(model.state_dict())
        print(f"New best accuracy: {best_acc:.4f} at epoch {epoch+1}, saving model.")
# save_path = "./No_glove_1250.pth"
save_path = "./glove_1250_skipLSTM.pth"
torch.save(best_model_state, save_path)

Epoch 1: loss=0.6946, acc=0.5000, f1=0.3598
New best accuracy: 0.5000 at epoch 1, saving model.
Epoch 2: loss=0.6939, acc=0.5000, f1=0.3333
Epoch 3: loss=0.6930, acc=0.4980, f1=0.3525
Epoch 4: loss=0.6930, acc=0.5020, f1=0.3480
New best accuracy: 0.5020 at epoch 4, saving model.
Epoch 5: loss=0.6917, acc=0.5060, f1=0.3499
New best accuracy: 0.5060 at epoch 5, saving model.
Epoch 6: loss=0.6903, acc=0.5060, f1=0.3565
Epoch 7: loss=0.6861, acc=0.5000, f1=0.3437
Epoch 8: loss=0.6807, acc=0.5020, f1=0.3513
Epoch 9: loss=0.6841, acc=0.5060, f1=0.3597
Epoch 10: loss=0.6880, acc=0.5040, f1=0.3649
Epoch 11: loss=0.6865, acc=0.5100, f1=0.3649
New best accuracy: 0.5100 at epoch 11, saving model.
Epoch 12: loss=0.6828, acc=0.5060, f1=0.3565
Epoch 13: loss=0.6806, acc=0.5040, f1=0.3679
Epoch 14: loss=0.7101, acc=0.4980, f1=0.3428
Epoch 15: loss=0.6858, acc=0.5000, f1=0.3437
Epoch 16: loss=0.6835, acc=0.5040, f1=0.3490
Epoch 17: loss=0.6749, acc=0.5040, f1=0.3523
Epoch 18: loss=0.6866, acc=0.5020, 

KeyboardInterrupt: 

In [ ]:
# best_model_state
# pre trained glove embed 0.6720 1250 samples 10e-3
# pre trained glove embed 0.6336 1250 samples 10e-3
# no pretrain embed 0.6112 1250 samples 10e-3
# 

# pre trained glove embed 0.7120 0.7260 500 samples 10e-3 128 hidden size

In [62]:
save_path = "./No_glove_1250.pth"
torch.save(best_model_state, save_path)

In [24]:
from torch.nn.utils.rnn import pad_sequence

MAX_LEN = 300

def collate_batch(batch):
    texts, labels = zip(*batch)
    # truncate or pad each text
    texts_truncated = [t[:MAX_LEN] for t in texts]
    texts_padded = pad_sequence(texts_truncated, batch_first=True, padding_value=vocab["<pad>"])
    # If some sequences are shorter than MAX_LEN, pad to MAX_LEN
    if texts_padded.shape[1] < MAX_LEN:
        pad_size = MAX_LEN - texts_padded.shape[1]
        padding = torch.full((texts_padded.shape[0], pad_size), vocab["<pad>"], dtype=torch.long)
        texts_padded = torch.cat([texts_padded, padding], dim=1)
    return texts_padded, torch.tensor(labels)

train_loader_2 = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=collate_batch)
test_loader_2 = DataLoader(test_data, batch_size=32, shuffle=False, collate_fn=collate_batch)

In [25]:

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPool1D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
seq_len = 34
embed_dim = 468 #300
num_classes = 2
num_epochs = 100
train_loss_arr, test_loss_arr, train_acc_arr, test_acc_arr = [], [] , [], []
model = Sequential([
    Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(MAX_LEN, 1)),
    MaxPool1D(pool_size=2),
    Conv1D(filters=256, kernel_size=5, activation='relu'),
    MaxPool1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

# model = Sequential([
#     # Input shape for Conv1D is (timesteps, features) = (seq_len, embed_dim)
#     Conv1D(filters=50, kernel_size=3, strides=1, padding='same',
#            activation='relu', input_shape=(983,)),#(seq_len, embed_dim)),
#     Conv1D(filters=75, kernel_size=3, strides=1, padding='same',
#            activation='relu'),
#     MaxPool1D(pool_size=2),                 # seq_len -> ceil(seq_len/2)
#     Dropout(0.25),

#     Conv1D(filters=125, kernel_size=3, strides=1, padding='same',
#            activation='relu'),
#     MaxPool1D(pool_size=2),                 # -> ceil(previous/2)
#     Dropout(0.25),

#     Flatten(),                              # → 1D vector
#     Dense(500, activation='relu'),
#     Dropout(0.4),
#     Dense(250, activation='relu'),
#     Dropout(0.3),
#     Dense(num_classes, activation='softmax')
# ])

model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.compile(optimizer=Adam(),
              loss='sparse_categorical_crossentropy',  # labels from loader are integer class IDs
              metrics=['accuracy'])

#model.summary()

# --- manual training loop using DataLoader batches ---
EPOCHS = 100
for epoch in range(EPOCHS):
    train_loss, train_acc = 0.0, 0.0
    batches = 0
    for xb, yb in train_loader_2:
        # convert torch tensors to numpy for Keras
        xb_np = xb.numpy()
        yb_np = yb.numpy()

        metrics = model.train_on_batch(xb_np, yb_np)
        train_loss += metrics[0]
        train_acc += metrics[1]
        batches += 1
    print(f"Epoch {epoch+1}: loss={train_loss/batches:.4f}, acc={train_acc/batches:.4f}")
    train_acc_arr.append(train_acc)
    train_loss_arr.append(train_loss/batches)
    # simple validation pass
    val_loss, val_acc, val_batches = 0.0, 0.0, 0
    for xb, yb in test_loader_2:
        xb_np = xb.numpy()
        yb_np = yb.numpy()
        metrics = model.test_on_batch(xb_np, yb_np)
        val_loss += metrics[0]
        val_acc += metrics[1]
        val_batches += 1
    print(f"           val_loss={val_loss/val_batches:.4f}, val_acc={val_acc/val_batches:.4f}")
    test_acc_arr.append(train_acc)
    test_loss_arr.append(train_loss/batches)

c:\Users\ASUS\anaconda3\envs\SC4001\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1: loss=528.6711, acc=0.5027
           val_loss=189.2079, val_acc=0.4907
Epoch 2: loss=114.3804, acc=0.4927
           val_loss=82.1652, val_acc=0.5090
Epoch 3: loss=64.2405, acc=0.5336
           val_loss=52.8107, val_acc=0.5450
Epoch 4: loss=44.8637, acc=0.5622
           val_loss=39.0375, val_acc=0.5716
Epoch 5: loss=34.5627, acc=0.5890
           val_loss=31.0410, val_acc=0.5980
Epoch 6: loss=28.1773, acc=0.6105
           val_loss=25.8254, val_acc=0.6172
Epoch 7: loss=23.8361, acc=0.6262
           val_loss=22.1560, val_acc=0.6318
Epoch 8: loss=20.6980, acc=0.6382
           val_loss=19.4398, val_acc=0.6419
Epoch 9: loss=18.3261, acc=0.6480
           val_loss=17.3502, val_acc=0.6522
Epoch 10: loss=16.4703, acc=0.6575
           val_loss=15.6876, val_acc=0.6600
Epoch 11: loss=14.9737, acc=0.6637
           val_loss=14.3353, val_acc=0.6663
Epoch 12: loss=13.7463, acc=0.6698
           val_loss=13.2147, val_acc=0.6717
Epoch 13: loss=12.7203, acc=0.6748
           val_loss=12.

In [ ]:
#epoch 100 0.70